In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

### get data from server

In [2]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [3]:
query = "SELECT product_characteristics.id, Brand_id , Detail, Summary, Brand FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows[0]

(1,
 1,
 '*Type: Shirt  \n*Fit: Regular fit  \n*Sleeve style: Long sleeve  \n*Neckline: Not applicable  \n*Material: Silk  \n*Formality: Formal  \n*Season: Spring, Summer  \n*Colors: 80% Pink, 20% Light Pink  \n*Texture: Smooth  \n*Transparency: Semi-transparent  \n*Details and Embellishments: Buttons  \n*Shape: Fitted  \n*Length: Hip-length  \n*Collar Style: Button-down  \n*Sleeve Style: Long  \n*Patterns: Geometric  \n*Patterns placement: Not applicable  \n*Fluidity of fabric: High  \n*Fabric weight: Light  \n*Pocket Presence: No  \n*Pocket placement: Not applicable  \n*Pocket size: Not applicable  \n*Breathability: High  \n*Occasion Suitability: Formal  \n*Lapel: Not applicable',
 'The Gucci silk shirt features the iconic GG monogram in a shadowed pattern, creating a subtle yet sophisticated texture. The shirt is crafted from high-quality silk, ensuring a fluid drape and lightweight feel, which enhances its breathability and comfort. The fit appears to be relaxed, offering a modern 

In [4]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [5]:
query = "SELECT Brand_id, base64 FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows_img

In [6]:
len(rows_img)

629

In [7]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

### organize data

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(rows ,columns= ['Id','prod_id','descripcion', 'Summary','Brand'])

In [10]:
df.head()

,Id,prod_id,descripcion,Summary,Brand
0,1,1,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The Gucci silk shirt features the iconic GG mo...,Gucci
1,2,2,*Type: Shirt\n*Fit: Regular fit\n*Sleeve style...,The Gucci Oxford cotton shirt with GG Shadow f...,Gucci
2,3,3,"*Type: Top, Shirt\n*Fit: Regular fit\n*Sleeve ...",The Gucci top depicted in the images is a crop...,Gucci
3,4,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The Gucci silk shirt with the Morsetto and str...,Gucci
4,5,5,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The shirt in the images is a Gucci cotton Oxfo...,Gucci


In [11]:
df['descripcion'] = df['descripcion'].str.replace('\n*', ' / ')
df['descripcion'] = df['descripcion'].str.replace(r'\s+', ' ')
df['descripcion'] = df['descripcion'].str.replace('*', ' ')

df.head(5)

,Id,prod_id,descripcion,Summary,Brand
0,1,1,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci silk shirt features the iconic GG mo...,Gucci
1,2,2,Type: Shirt / Fit: Regular fit / Sleeve style...,The Gucci Oxford cotton shirt with GG Shadow f...,Gucci
2,3,3,"Type: Top, Shirt / Fit: Regular fit / Sleeve ...",The Gucci top depicted in the images is a crop...,Gucci
3,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci silk shirt with the Morsetto and str...,Gucci
4,5,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt in the images is a Gucci cotton Oxfo...,Gucci


In [12]:
df['Complete_description'] = df['Summary'] + ' /' + "Brand: " + df['Brand'] + " /" + df['descripcion']
df.head()

,Id,prod_id,descripcion,Summary,Brand,Complete_description
0,1,1,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci silk shirt features the iconic GG mo...,Gucci,The Gucci silk shirt features the iconic GG mo...
1,2,2,Type: Shirt / Fit: Regular fit / Sleeve style...,The Gucci Oxford cotton shirt with GG Shadow f...,Gucci,The Gucci Oxford cotton shirt with GG Shadow f...
2,3,3,"Type: Top, Shirt / Fit: Regular fit / Sleeve ...",The Gucci top depicted in the images is a crop...,Gucci,The Gucci top depicted in the images is a crop...
3,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci silk shirt with the Morsetto and str...,Gucci,The Gucci silk shirt with the Morsetto and str...
4,5,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt in the images is a Gucci cotton Oxfo...,Gucci,The shirt in the images is a Gucci cotton Oxfo...


In [13]:
documents = df['Complete_description'].tolist()
id = df['Id'].astype('str').tolist()
metadata = [{"source": id} for id in df['Id'].astype('str').tolist()]

In [14]:
df_image = pd.DataFrame(rows_img, columns=['prod_id', 'base64'])

In [15]:
df_image.head(2)

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [16]:
df_image.shape

(629, 2)

In [17]:
del rows, rows_img

### Get images 

In [18]:
import base64
from io import BytesIO
from PIL import Image

In [19]:
def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]

    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

In [20]:
def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

In [21]:
def resize_image(image, new_width):
    # Open the image

    # Calculate the new height while maintaining the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

    return resized_image


In [23]:
df_image.head()

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
4,2,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [24]:
df_image.iloc[42]

prod_id                                                   11
base64     data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...
Name: 42, dtype: object

### Testing embeddings

In [25]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [26]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [27]:
embedding_function = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

### organized text 

In [28]:
df = pd.merge(
            df,
            df_image.groupby('prod_id')['base64'].apply(list).reset_index(),
            on='prod_id'
        )

In [29]:
df.prod_id = df.prod_id.astype(int)
df.sort_values('prod_id',ascending=True,inplace=True)
df.reset_index(drop=True, inplace= True)
df.prod_id = df.prod_id.astype('string')
df.head(5)

,Id,prod_id,descripcion,Summary,Brand,Complete_description,base64
0,1,1,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci silk shirt features the iconic GG mo...,Gucci,The Gucci silk shirt features the iconic GG mo...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,2,2,Type: Shirt / Fit: Regular fit / Sleeve style...,The Gucci Oxford cotton shirt with GG Shadow f...,Gucci,The Gucci Oxford cotton shirt with GG Shadow f...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,3,3,"Type: Top, Shirt / Fit: Regular fit / Sleeve ...",The Gucci top depicted in the images is a crop...,Gucci,The Gucci top depicted in the images is a crop...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci silk shirt with the Morsetto and str...,Gucci,The Gucci silk shirt with the Morsetto and str...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,5,5,Type: Shirt / Fit: Regular fit / Sleeve s...,The shirt in the images is a Gucci cotton Oxfo...,Gucci,The shirt in the images is a Gucci cotton Oxfo...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


### chunks

In [30]:
df = df[['prod_id', 'Summary','Brand','base64']]._append(df[['prod_id', 'descripcion','Brand','base64']].rename(columns = {'descripcion':'Summary'}),ignore_index=True)

In [31]:
df.Brand.unique()

array(['Gucci', 'HM', 'Prada', 'Mango'], dtype=object)

In [65]:
df['Type'] = df['Brand'].apply(lambda x: 'Luxury' if (x == 'Gucci') or (x == 'Prada') else 'Retail')

In [66]:
df.head()

,prod_id,Summary,Brand,base64,Type
0,1,The Gucci silk shirt features the iconic GG mo...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
1,2,The Gucci Oxford cotton shirt with GG Shadow f...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
2,3,The Gucci top depicted in the images is a crop...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
3,4,The Gucci silk shirt with the Morsetto and str...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
4,5,The shirt in the images is a Gucci cotton Oxfo...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury


In [67]:
persist_directory = '../../../Chromadb/chroma-retail'

In [68]:
store = InMemoryStore()
id_key = 'doc_id'

In [69]:
min(df['base64'].apply(lambda x: len(x)))

1

In [70]:
df[ df['base64'].apply(lambda x: len(x)) == 1 ]

,prod_id,Summary,Brand,base64,Type
130,131,The shirt is a fluid fabric top with a straigh...,Mango,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Retail
284,131,"Type: Top, Shirt / Fit: Regular fit / Sle...",Mango,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Retail


In [71]:
doc_ids = df.prod_id


texts = [
    Document(
        page_content= df.iloc[i].Summary, 
        metadata={
            id_key:df.iloc[i].prod_id,
            'Type': df.iloc[i].Type
            }
    )
    for i in df.index
]


In [111]:
texts[0]

Document(page_content='The Gucci silk shirt features the iconic GG monogram in a shadowed pattern, creating a subtle yet sophisticated texture. The shirt is crafted from high-quality silk, ensuring a fluid drape and lightweight feel, which enhances its breathability and comfort. The fit appears to be relaxed, offering a modern and versatile silhouette suitable for various occasions, from formal to semi-formal events. It has long sleeves with buttoned cuffs, adding a touch of classic elegance, and a traditional pointed collar that complements its refined style. The shirt\'s primary color is a rich, deep burgundy, referred to as "Wine Red," which is accentuated by the shadowed GG pattern in a slightly darker shade, creating a monochromatic yet dynamic visual effect. The material\'s natural sheen adds a luxurious finish to the overall look. This shirt is ideal for cooler seasons due to its long sleeves and silk\'s insulating properties. There are no visible pockets, maintaining a sleek an

In [73]:
texts[0].metadata['doc_id']

'1'

In [61]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20,)

In [120]:
text_child = []

for i in df.index:

    child_docs=child_splitter.create_documents(
            [texts[i].page_content],
            metadatas= 
                [{
                    'doc_id':texts[i].metadata['doc_id'],
                    'Type': df.iloc[i].Type
                }]
        )
    
    for _doc in child_docs:
        text_child.append(_doc)
    

In [75]:
vectorstore = Chroma.from_documents(
                documents=texts,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='RAG-Parent',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

In [118]:
vectorstore.similarity_search('HM')[0].page_content

"The shirt is a fluid, semi-transparent black top with a straight fit, providing a relaxed yet elegant silhouette. It features long sleeves with buttoned cuffs, adding a touch of sophistication suitable for both formal and semi-formal occasions. The neckline is a classic round neck, complementing the simplicity of the design. The material appears lightweight and breathable, making it ideal for transitional seasons like spring and fall. The fabric's fluidity enhances the garment's drape, ensuring a flattering fit on various body types. The shirt is primarily black, a versatile and timeless color that can be easily paired with various bottoms and accessories. There are no visible pockets, patterns, or embellishments, maintaining a clean and minimalist aesthetic."

In [122]:
vectorstore = Chroma.from_documents(
                documents=text_child,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='RAG-Child',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

In [130]:
vectorstore.similarity_search_with_score('Luxury')

[(Document(page_content='occasions, making it a stylish addition to any wardrobe.', metadata={'Type': 'Retail', 'doc_id': '103'}),
  1.2316887378692627),
 (Document(page_content='sense of luxury and is versatile for various occasions. The fabric appears lightweight and breathable, making it suitable for both formal and semi-formal events, especially in transitional seasons like spring and fall. There are no visible pockets, patterns, or embellishments, keeping the design clean and minimalist. The overall style is chic and understated, perfect for pairing with tailored', metadata={'Type': 'Retail', 'doc_id': '121'}),
  1.2735806703567505),
 (Document(page_content='or additional embellishments, maintaining a simple yet refined aesthetic. Its fluid fabric drapes elegantly, making it suitable for casual daytime outings or relaxed social gatherings.', metadata={'Type': 'Luxury', 'doc_id': '9'}),
  1.288376808166504),
 (Document(page_content='The garment in the images is a fluid, satin-like 

### Already loaded !!!

In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={"k":5}
)

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
retriever.vectorstore.add_documents(texts)
#retriever.docstore.mset(list(zip(doc_ids, texts)))

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
df.iloc[1].Complete_description

### Talk to Chroma

In [ ]:
from apikey import apikey 
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] = apikey

In [ ]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
system_prompt = (
    "You are a Fashion specialist engine to sugest clothing pieces from context. "
    "If you don't know the answer to the question, say that you dont know "
    "answer concise."
    #"Explain why the selected options of the context are the best ones"
    "\n\n"
    "{context}"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | turbo_llm
    | StrOutputParser()
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
query = "Im looking for clothing peaces from Mango that could be similar to Gucci clothing"

In [ ]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

In [ ]:
for document in response["context"]:
    print(document)
    print()

In [ ]:
df_image[df_image.prod_id == 41]

In [ ]:
image_bytes = base64_to_image(df_image.iloc[113].base64)
img = create_image_from_bytes(image_bytes=image_bytes)
img.resize((400,500))